In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

f = fits.open('/home/regier/bliss/data/sdss/2583/2/136/frame-r-002583-2-0136.fits')
w = WCS(f[0].header)

# lower-left corner of the 100x100-pixel study area is at pixel (310, 630)
w.pixel_to_world(310, 630)

In [ ]:
import pandas as pd

# catalog from https://catalogs.mast.stsci.edu/hsc/
# target: 323.3357504,-0.807026
# radius of 32 arcsec; v3; detailed; MagAuto <= 24
fn = '/home/regier/bliss/case_studies/dependent_tiling/HSC-10_20_2023.csv'
hst_catalog = pd.read_csv(fn)
hst_catalog

In [ ]:
from astropy.coordinates import SkyCoord

hst_coords = SkyCoord(ra=hst_catalog['MatchRA'], dec=hst_catalog['MatchDec'], unit='deg')
hst_pix = w.world_to_pixel(hst_coords)

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(f[0].data, origin='lower', cmap='gray_r')

In [ ]:
from matplotlib.patches import Rectangle

plt.imshow(f[0].data, origin='lower', cmap='gray_r')
plt.scatter(hst_pix[0], hst_pix[1], s=10, c='r')
rect = Rectangle((310, 630), 100, 100, linewidth=1, edgecolor='b', facecolor='none')
plt.gca().add_patch(rect)


In [ ]:
in_bounds = (hst_pix[0] > 310) & (hst_pix[0] < 410) & (hst_pix[1] > 630) & (hst_pix[1] < 730)
hst_inbounds = hst_catalog[in_bounds]

In [ ]:
plt.imshow(f[0].data, origin='lower', cmap='gray_r')
plt.scatter(hst_pix[0][in_bounds], hst_pix[1][in_bounds], s=10, c='r')
rect = Rectangle((310, 630), 100, 100, linewidth=1, edgecolor='b', facecolor='none')
plt.gca().add_patch(rect)

In [ ]:
hst_inbounds["MatchID"].unique().shape

In [ ]:
hst_inbounds

In [ ]:
from os import environ
environ["CUDA_VISIBLE_DEVICES"] = "1"

from pathlib import Path
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from bliss.encoder import Encoder


environ["BLISS_HOME"] = str(Path().resolve().parents[1])
with initialize(config_path="../../case_studies/dependent_tiling/", version_base=None):
    cfg = compose("config", {"encoder.two_layers=true"})

encoder: Encoder = instantiate(cfg.encoder)
# should use a model trained with aligned data, or with only one band
encoder.load_state_dict(torch.load("../../case_studies/dependent_tiling/m2.pt"))
encoder.eval()
encoder.cuda()
torch.set_grad_enabled(False)

In [ ]:
dataset = instantiate(cfg.predict.dataset, load_image_data=True)
frame = dataset[0]
batch = {
    "images": torch.from_numpy(frame["image"]).unsqueeze(0).cuda(),
    "background": torch.from_numpy(frame["background"]).unsqueeze(0).cuda()
}

In [ ]:
batch["images"].shape

In [ ]:
est_cat, pred = encoder.predict_step(batch, None).values()
est_full = est_cat.to_full_catalog()
est_full.n_sources.sum()

In [ ]:
from bliss.metrics import BlissMetrics, MetricsMode

metrics = BlissMetrics(
    mode=MetricsMode.FULL, slack=1.0, survey_bands=[0, 1, 2, 3, 4]
).cuda()

In [ ]:
metrics(est_full, est_full)

In [ ]:
hcat = hst_inbounds[hst_inbounds.Filter == "F606W"]
hcat

In [ ]:
ra = torch.from_numpy(hcat.MatchRA.values)
dec = torch.from_numpy(hcat.MatchRA.values)
mag = torch.from_numpy(hcat.MagAuto.values)

In [ ]:
mag

In [ ]:
from bliss.catalog import FullCatalog

plocs = FullCatalog.plocs_from_ra_dec(ra, dec, frame["wcs"][2])
plocs.shape

In [ ]:
d = {
    "plocs": plocs.unsqueeze(0),
    "star_fluxes": mag.unsqueeze(0).unsqueeze(2),
    "n_sources": torch.tensor(plocs.shape[0]).unsqueeze(0),
    "source_type": torch.zeros(plocs.shape[0]).unsqueeze(0).unsqueeze(2).long(),
}
true_cat = FullCatalog(112, 112, d).to("cuda")

In [ ]:
metric = metrics(est_full, true_cat)

In [ ]:
metric["detection_recall"]